In [5]:
import pandas as pd

# Widths of the 13 columns based on IPUMS codebook table
column_widths = [4, 6, 8, 13, 10, 13, 12, 1, 1, 2, 1, 5, 7]

# Column names based on the IPUMS codebook
column_names = [
    "YEAR", "SAMPLE", "SERIAL", "CBSERIAL", "HHWT", "CLUSTER", "STRATA", 
    "GQ", "OWNERSHP", "OWNERSHPD", "MORTGAGE", "MORTAMT1", "HHINCOME"
]

# load 30 rows to explore dataset
file_path = '../../data/usa_00001.dat'
df = pd.read_fwf(file_path, widths=column_widths, nrows=30, names=column_names)

df.head(30)


,YEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STRATA,GQ,OWNERSHP,OWNERSHPD,MORTGAGE,MORTAMT1,HHINCOME
0,2008,200801,1,23,5700,2008000000011,130001,1,1,13,3,980,58100
1,2008,200801,1,23,5700,2008000000011,130001,1,1,13,3,980,58100
2,2008,200801,1,23,5700,2008000000011,130001,1,1,13,3,980,58100
3,2008,200801,1,23,5700,2008000000011,130001,1,1,13,3,980,58100
4,2008,200801,2,39,11300,2008000000021,90101,1,1,13,3,1200,125000
5,2008,200801,2,39,11300,2008000000021,90101,1,1,13,3,1200,125000
6,2008,200801,3,70,10100,2008000000031,90401,1,1,12,1,0,50000
7,2008,200801,3,70,10100,2008000000031,90401,1,1,12,1,0,50000
8,2008,200801,3,70,10100,2008000000031,90401,1,1,12,1,0,50000
9,2008,200801,4,122,8300,2008000000041,110001,1,1,13,3,690,45480


Removing duplicates fomr the whole dataset with benchmark

In [6]:
import os
import time

In [7]:
# Define file path, column widths, and names
file_path = '../../data/usa_00001.dat'
column_widths = [4, 6, 8, 13, 10, 13, 12, 1, 1, 2, 1, 5, 7]  # Adjust based on your actual structure
column_names = ['YEAR', 'SAMPLE', 'SERIAL', 'CBSERIAL', 'HHWT', 'CLUSTER', 'STRATA', 'GQ', 'OWNERSHP', 'OWNERSHPD', 'MORTGAGE', 'MORTAMT1', 'HHINCOME']

# Benchmark start
start_time = time.time()

# Initialize the file for cleaned data and set a chunk size
cleaned_file_path = '../../data/processed_data.csv'
chunk_size = 75000  # Adjust based on system memory

# Initialize a set to track unique household identifiers (e.g., SERIAL + SAMPLE)
unique_households = set()

# Open a CSV file to write cleaned data
with open(cleaned_file_path, 'w') as f:

    # Process the file in chunks
    for chunk in pd.read_fwf(file_path, widths=column_widths, names=column_names, chunksize=chunk_size):
        
        # Drop duplicates within the chunk based on SERIAL and SAMPLE
        chunk_cleaned = chunk.drop_duplicates(subset=['SERIAL', 'SAMPLE'])

        # Create a unique identifier for households (SERIAL + SAMPLE) for efficient comparison
        chunk_cleaned['unique_id'] = chunk_cleaned.apply(lambda row: (row['SERIAL'], row['SAMPLE']), axis=1)

        # Filter out rows that have already been processed in previous chunks
        new_rows = chunk_cleaned[~chunk_cleaned['unique_id'].isin(unique_households)]

        # Update the set with new unique household identifiers
        unique_households.update(new_rows['unique_id'])

        # Drop the 'unique_id' column before saving and ensure column order
        new_rows = new_rows.drop(columns=['unique_id'])[column_names]

        # Append the cleaned chunk to the CSV file
        new_rows.to_csv(f, index=False, header=f.tell() == 0)  # Only write header if it's the first chunk

# Step 2: Measure time and output benchmark results
total_time = time.time() - start_time
print(f"Time to process and drop duplicates: {total_time:.2f} seconds")


/var/folders/14/bh3qw6yj6g3f5z7sdk9kyb6w0000gn/T/ipykernel_51738/1935585219.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk_cleaned['unique_id'] = chunk_cleaned.apply(lambda row: (row['SERIAL'], row['SAMPLE']), axis=1)
/var/folders/14/bh3qw6yj6g3f5z7sdk9kyb6w0000gn/T/ipykernel_51738/1935585219.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  chunk_cleaned['unique_id'] = chunk_cleaned.apply(lambda row: (row['SERIAL'], row['SAMPLE']), axis=1)
/var/folders/14/bh3qw6yj6g3f5z7sdk9kyb6w0000gn/T/ipy

Time to process and drop duplicates: 21544.76 seconds
